In [2]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta

In [3]:
DIR_CSV = "C:\\Users\\tora2\\IdeaProjects\\LifeBehavior\\logs\\behavior_log.csv"

In [4]:
BEHAVIOR_TYPE_TO_ACTIVITY = {
    "SLEEP": "睡眠",
    "EATING": "食事",
    "PERSONAL_CARE": "身のまわりの用事",
    "MEDICAL_CARE": "療養・静養",
    "WORK": "仕事",
    "WORK_SOCIALIZING": "仕事のつきあい",
    "SCHOOL_ACTIVITIES": "授業・学内の活動",
    "STUDY_OUTSIDE": "学校外の学習",
    "HOUSEWORK": "炊事・掃除・洗濯",
    "SHOPPING": "買い物",
    "CHILDCARE": "子どもの世話",
    "HOUSEHOLD_TASKS": "家庭雑事",
    "COMMUTING": "通勤",
    "SCHOOL_COMMUTING": "通学",
    "SOCIAL_PARTICIPATION": "社会参加",
    "SOCIALIZING": "会話・交際",
    "SPORTS": "スポーツ",
    "RECREATION": "行楽・散策",
    "HOBBIES": "趣味・娯楽・教養",
    "MEDIA_CONSUMPTION": "マスメディア接触",
    "REST": "休息"
}
# 各アクティビティに対して色を指定
COLOR_MAP = {
    '食事': '#5DAE8B',  # 濃いめの柔らかいグリーン
    '授業・学内の活動': '#E59866',  # 濃いめの柔らかいオレンジ
    '買い物': '#F7DC6F',  # 濃いめのイエロー
    '会話・交際': '#5499C7',  # 濃いめの柔らかいブルー
    'マスメディア接触': '#AF7AC5',  # 濃いめのパープル
    '仕事': '#F4D03F',  # 濃いライトゴールド
    '療養・静養': '#EC7063',  # 濃いめのピンク
    '子どもの世話': '#5DADE2',  # 濃いライトブルー
    '行楽・散策': '#DC7633',  # 濃いめのオレンジ
    '休息': '#58D68D',  # 濃いめのグリーン
    '睡眠': '#28B463',  # 濃いめのグリーン
    '通勤': '#85C1E9',  # 濃いめのブルー
    'スポーツ': '#F1C40F',  # 濃いめのライトイエロー
    '身のまわりの用事': '#E74C3C',  # 濃いめのレッド
    '家庭雑事': '#48C9B0',  # 濃いめのアクア
    '通学': '#A569BD',  # 濃いラベンダー
    '炊事・掃除・洗濯': '#F1948A',  # 濃いめのピンクベージュ
    '趣味・娯楽・教養': '#F5B041',  # 濃いゴールド
    '仕事のつきあい': '#85929E',  # 濃いめのグレー
    '社会参加': '#F8C471',  # 濃いめのライトオレンジ
    '学校外の学習': '#76D7C4'  # 濃いめのアクア
}
ACTIVITY_ORDERING = [
            "睡眠", "食事", "身のまわりの用事", "療養・静養", "仕事", "仕事のつきあい", "授業・学内の活動",
            "学校外の学習", "炊事・掃除・洗濯", "買い物", "子どもの世話", "家庭雑事", "通勤", "通学", "社会参加",
            "会話・交際", "スポーツ", "行楽・散策", "趣味・娯楽・教養", "マスメディア接触", "休息"]

def parse_current_time_to_datetime(time_str):
    # "0/00:00:00" 形式の文字列をパース
    day_part, time_part = time_str.split('/')  # 例: "0", "00:00:00"
    hours, minutes, seconds = map(int, time_part.split(':'))  # 時、分、秒をintに変換
    days = int(day_part)  # 日数部分をintに変換

    # 基準日（例えば、今日の日付）からの timedelta を計算
    base_date = datetime(2024, 9, 1)  # 基準日を設定（任意の日付を使用）
    delta = timedelta(days=days, hours=hours, minutes=minutes, seconds=seconds)
    
    # 基準日に timedelta を加えて最終的な datetime を計算
    result_datetime = base_date + delta
    return result_datetime

In [5]:
result = pd.read_csv(DIR_CSV).replace(BEHAVIOR_TYPE_TO_ACTIVITY)
weekday = result.query("CurrentDay not in ('SUNDAY','SATURDAY')").drop(columns=["CurrentDay"])
saturday = result.query("CurrentDay == 'SATURDAY'").drop(columns=["CurrentDay"])
sunday = result.query("CurrentDay == 'SUNDAY'").drop(columns=["CurrentDay"])
allday = result.drop(columns=["CurrentDay"])

In [41]:
def generate_html_schedule_with_fixed_headers_and_fullscreen(allday: DataFrame, color_map, activity_ordering):
    # HTMLのヘッダーとCSS
    html_content = '''
    <!DOCTYPE html>
    <html lang="ja">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>エージェントのタイムスケジュール</title>
        <link rel="stylesheet" href="https://fonts.googleapis.com/css2?family=Noto+Sans&display=swap">
        <style>
            body {
                font-family: 'Noto Sans', sans-serif;
                margin: 0;
                padding: 0;
                height: 100vh;
                display: flex;
                flex-direction: column;
            }
            h1 {
                text-align: center;
                padding: 10px 0;
                background-color: #2c3e50;
                color: #ecf0f1;
                font-size: 10px;
                margin: 0;
                letter-spacing: 1px;
            }
            .table-container {
                flex: 1;
                padding: 10px;
                display: flex;
                overflow: hidden; /* スクロール領域を制限 */
            }
            .scroll-table {
                width: 100%;
                height: 100%;
                overflow: auto; /* スクロールを有効に */
                border: 1px solid #ddd;
                box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
                display: block;
            }
            table {
                border-collapse: collapse;
                width: 100%;
                table-layout: fixed;
                min-width: 100%;
                height: 100%;
                background-color: white;
            }
            th, td {
                border: 1px solid #ddd;
                padding: 10px;
                text-align: center;
                vertical-align: middle;
                white-space: nowrap;
                width: 20px; /* 各列の幅を固定 */
            }
            th {
                background-color: #34495e;
                color: white;
                text-transform: uppercase;
                font-size: 10px;
                letter-spacing: 1px;
                position: sticky;
                top: 0; /* 上部の固定 */
                z-index: 1;
            }
            td {
                font-size: 10px;
                transition: background-color 0.3s;
                font-weight: bold;
                color: #333;
                border-radius: 4px;
            }
            /* 左側（時間）の固定 */
            td:first-child, th:first-child {
                position: sticky;
                left: 0;
                z-index: 2;
                background-color: #34495e;
                color: white;
            }
            .time-slot {
                height: 15px;
            }
            /* 各行動に応じた蛍光色の定義 */
    '''

    # 蛍光色の定義をCSSに追加
    for activity, color in color_map.items():
        html_content += f'''
        .{activity} {{
            background-color: {color};
            color: white;
        }}
        '''

    html_content += '''
        </style>
    </head>
    <body>

    <h1>エージェントのタイムスケジュール</h1>

    <div class="table-container">
        <div class="scroll-table">
            <table>
                <thead>
                    <tr>
                        <th>時間</th>
    '''

    # エージェントの列（カラム）ヘッダーを追加
    agents = allday.columns[1:]  # CurrentTime以外のカラムを取得
    for agent in agents:
        html_content += f'<th>{agent}</th>'

    html_content += '''
                    </tr>
                </thead>
                <tbody>
    '''

    # データをHTMLテーブルに変換 (セル結合処理)
    row_span_tracker = {agent: 0 for agent in agents}  # 各エージェントのセル結合数をトラッキング
    row_span_activity = {agent: None for agent in agents}  # 各エージェントの現在の行動を記録

    for index, row in allday.iterrows():
        html_content += '<tr>'
        html_content += f'<td>{row["CurrentTime"]}</td>'  # 時間

        for agent in agents:
            activity = row[agent]

            # 現在の行動が前の行と異なる場合、新しいセルを作成
            if row_span_tracker[agent] == 0:
                # 次の行動が同じかどうか確認
                rowspan = 1
                for next_index in range(index + 1, len(allday)):
                    next_activity = allday.iloc[next_index][agent]
                    if next_activity == activity:
                        rowspan += 1
                    else:
                        break

                # セルを結合する場合（rowspanが2以上）、トラッカーを更新
                if rowspan > 1:
                    html_content += f'<td class="{activity}" rowspan="{rowspan}">{activity}</td>'
                    row_span_tracker[agent] = rowspan - 1
                else:
                    html_content += f'<td class="{activity}">{activity}</td>'
            else:
                # 結合済みセルはスキップ
                row_span_tracker[agent] -= 1

        html_content += '</tr>'

    html_content += '''
                </tbody>
            </table>
        </div>
    </div>

    </body>
    </html>
    '''
    return html_content


In [47]:
# HTML生成部分

html_output = generate_html_schedule_with_fixed_headers_and_fullscreen(allday.iloc[0:96*2,range(0,1000,25)], COLOR_MAP, ACTIVITY_ORDERING)

# HTMLファイルに書き込み
with open("C:\\Users\\tora2\\IdeaProjects\\LifeBehavior\\visualization\\html\\individual_schdule.html", "w", encoding="utf-8") as file:
    file.write(html_output)

print("HTMLファイルが作成されました")

HTMLファイルが作成されました
